# Joern-ey into Metagol

Old, broken metagol generation script below:

In [2]:
import pandas as pd

In [3]:
ilp_data = pd.read_csv("../data/ilp_dataset.csv.gz")
ilp_data = ilp_data.drop('Unnamed: 0', axis='columns')
ilp_data

In [5]:
prolog = pd.read_csv("../data/ilp_prolog_data.csv.gz")
prolog = prolog.drop('Unnamed: 0', axis='columns')
prolog

,testcase_ID,flaw,bug,code_length,tree,source_map
0,-232086,CWE-122,True,1625,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_232086_id_1_f_l_c_, ""p..."
1,-232012,CWE-122,True,1619,% START: Generated Prolog\n% NODE PROPERTIES \...,% CODE\nsource_code(bad_232012_id_1_f_l_48_c_1...
2,-62917,CWE-121,True,1622,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_62917_id_0_f_l_c_, ""p1..."
3,-62901,CWE-121,True,1627,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_62901_id_0_f_l_c_, ""p1..."
4,-62900,CWE-121,True,1576,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_62900_id_0_f_l_c_, ""p2..."
5,-62869,CWE-121,True,1616,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_62869_id_0_f_l_c_, ""p1..."
6,-62854,CWE-121,True,1630,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_62854_id_0_f_l_c_, ""p1..."
7,-62853,CWE-121,True,1621,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_62853_id_0_f_l_c_, ""p1..."
8,-62852,CWE-121,True,1570,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_62852_id_0_f_l_c_, ""p3..."
9,-62804,CWE-121,True,1618,% START: Generated Prolog\n% NODE PROPERTIES \...,"% CODE\nsource_code(bad_62804_id_0_f_l_c_, ""p1..."


In [6]:
good_example = prolog[prolog['testcase_ID'] == 62852].iloc[0]
bad_example = prolog[prolog['testcase_ID'] == -62852].iloc[0]

In [7]:
def extract_node_ids(tree):
    nodes = set()
    
    for line in tree.split('\n'):
        line = line.strip()
        if not line.startswith("%") and line:
            parent, child = line[4:-2].split(", ")
            nodes.add(parent)
            nodes.add(child)
            
    return nodes

In [8]:
good_nodes = extract_node_ids(good_example.tree)
bad_nodes = extract_node_ids(bad_example.tree)

ValueError: not enough values to unpack (expected 2, got 1)

In [9]:
positive_examples = [
    'bug('+node_id+')' for node_id in bad_nodes
]

negative_examples = [
    'bug('+node_id+')' for node_id in good_nodes
]

base_rules = [
]

meta_rules = [
    'metarule([P,Q],[P,A,B],[[Q,A,B]]).',
    'metarule([P,Q,R],[P,A,B],[[Q,A,B],[R,A,B]]).',
    'metarule([P,Q,R],[P,A,B],[[Q,A,C],[R,C,B]]).',
]

NameError: name 'bad_nodes' is not defined

In [10]:
script_template = """
:- use_module(library(metagol)).

%% metagol settings
{settings}

%% background knowledge
{background_knowledge}

%% metarules
{metarules}

%% learning task
:-
  %% positive examples
  Pos = [
    {positive_examples}
  ],
  %% negative examples
  Neg = [
    {negative_examples}
  ],
  learn(Pos,Neg).
"""

script = script_template.format(
    background_knowledge='\n'.join([good_example.tree, bad_example.tree] + base_rules),
    settings="""
    body_pred(ast/2).
    body_pred(cfg/2).
    """,
    metarules='\n'.join(meta_rules),
    positive_examples=',\n'.join(positive_examples),
    negative_examples=',\n'.join(negative_examples),
)
print(script)

NameError: name 'base_rules' is not defined

In [11]:
with open("test.pl", 'w') as f:
    f.write(script)

NameError: name 'script' is not defined